## imports

In [96]:
import polars as pl
from polars import col as c
import os, sys, requests, re
from dotenv import load_dotenv, find_dotenv
from tqdm import tqdm
from datetime import datetime

load_dotenv(find_dotenv())

True

## constants

In [97]:
OMDB_API_KEY = os.getenv("OMDB_API_KEY")
FILES_DIR = "../data"

## load df

In [98]:
file_name = "movies_df.parquet"
file_path = os.path.join(FILES_DIR, file_name)
movies_df = pl.read_parquet(file_path)

In [99]:
# file_name = "movies_df.xlsx"
# file_path = os.path.join(FILES_DIR, file_name)
# movies_df = pl.read_excel(file_path)

## custom funcs

In [100]:
def fetch_english_title(title):
    try:
        url = f"http://www.omdbapi.com/?apikey={OMDB_API_KEY}&i={requests.utils.quote(title)}"  # for index search
        # url = f"http://www.omdbapi.com/?apikey={OMDB_API_KEY}&t={requests.utils.quote(title)}"
        response = requests.get(url)
        data = response.json()
        if data.get("Response") == "True" and "Title" in data:
            return data
        else:
            print(f"Not found in OMDb: {title}")
            return "Not found"
    except Exception as e:
        print(f"OMDb error for {title}: {e}")
        return "Not found"

In [111]:
def add_row_by_id(
    df: pl.DataFrame,
    omdb_id: str,
    viewed_date: str = datetime.today().date(),
    default_values=None,
    liked: bool = False,
) -> pl.DataFrame:
    """
    Add a new row to the dataframe by specifying the 'id' value.
    Other columns will be filled with None or values from default_values dict.

    Args:
        df (pl.DataFrame): The dataframe to add the row to.
        id (str): The value for the 'id' column.
        default_values (dict, optional): Dictionary of column: value pairs to fill in.

    Returns:
        pl.DataFrame: DataFrame with the new row appended.
    """
    if default_values is None:
        default_values = {}
    # Prepare new row as dict
    new_row = {col: default_values.get(col, None) for col in df.columns}
    new_row["omdb_id"] = omdb_id
    new_row["viewed"] = viewed_date
    new_row["index"] = df["index"].max() + 1
    new_row["liked"] = liked

    creds = fetch_english_title(omdb_id)
    if creds == "Not found":
        print("ERROR: cannot upload creds!")
        raise ValueError("CredsNotFoundError")
    else:
        for col_name in [
            "title",
            "year",
            "genre",
            "director",
            "country",
            "actors",
            "box_office",
            "writer",
            "language",
            "imdb_rating",
        ]:
            new_row[col_name] = creds[
                "".join(
                    [
                        word.capitalize() if word != "imdb" else word
                        for word in col_name.split("_")
                    ]
                )
            ]
            if col_name == "box_office":
                try:
                    new_row[col_name] = int(re.sub(r"[^\d]", "", new_row[col_name]))
                except:
                    print(
                        f"the value for box_office is {new_row[col_name]}. Fill with null."
                    )
                    new_row[col_name] = None

    # Create single-row DataFrame
    new_row_df = pl.DataFrame([new_row])
    # Convert dtypes to match df
    new_row_df = new_row_df.cast(df.schema)
    # Concatenate
    return pl.concat([df, new_row_df], how="vertical")


# test_func
_ = add_row_by_id(
    df=movies_df,
    omdb_id="tt0038355",
    default_values=None,
)

## code

In [112]:
NEW_MOVIE_OMDB_ID = "tt0038787"

In [113]:
# check if everything is correct
display(
    add_row_by_id(
        df=movies_df,
        omdb_id=NEW_MOVIE_OMDB_ID,
        liked=False,
    ).tail()
)

# add check that movie was not viewed before
assert NEW_MOVIE_OMDB_ID not in movies_df["omdb_id"].unique()

the value for box_office is N/A. Fill with null.


index,title,year,viewed,liked,omdb_id,genre,director,country,actors,box_office,writer,language,imdb_rating
u32,str,i64,date,bool,str,str,str,str,str,i32,str,str,f32
622,"""The Notebook""",2004,2025-08-07,false,"""tt0332280""","""Drama, Romance""","""Nick Cassavetes""","""United States""","""Gena Rowlands, James Garner, R…",81417274,"""Jeremy Leven, Jan Sardi, Nicho…","""English""",7.8
623,"""The Accidental Tourist""",1988,2025-08-11,false,"""tt0094606""","""Comedy, Drama, Romance""","""Lawrence Kasdan""","""United States""","""William Hurt, Kathleen Turner,…",32632093,"""Anne Tyler, Frank Galati, Lawr…","""English""",6.7
624,"""Four Weddings and a Funeral""",1994,2025-08-12,false,"""tt0109831""","""Comedy, Drama, Romance""","""Mike Newell""","""United Kingdom""","""Hugh Grant, Andie MacDowell, J…",52700832,"""Richard Curtis""","""English, British Sign """,7.1
625,"""Rear Window""",1954,2025-08-15,false,"""tt0047396""","""Drama, Mystery, Thriller""","""Alfred Hitchcock""","""United States""","""James Stewart, Grace Kelly, We…",37622343,"""John Michael Hayes, Cornell Wo…","""English""",8.5
626,"""Notorious""",1946,2025-08-17,false,"""tt0038787""","""Drama, Film-Noir, Romance""","""Alfred Hitchcock""","""United States""","""Cary Grant, Ingrid Bergman, Cl…",null,"""Ben Hecht, Alfred Hitchcock, J…","""English, Portuguese, French""",7.9


In [114]:
print("init shape:\t", movies_df.shape[0])
movies_df = add_row_by_id(
    df=movies_df,
    omdb_id=NEW_MOVIE_OMDB_ID,
    liked=False,
)
print("result shape:\t", movies_df.shape[0])

init shape:	 625
the value for box_office is N/A. Fill with null.
result shape:	 626


## save df

In [115]:
file_name = "movies_df.parquet"
file_path = os.path.join(FILES_DIR, file_name)
movies_df.write_parquet(file_path)

In [116]:
file_name = "movies_df.xlsx"
file_path = os.path.join(FILES_DIR, file_name)
movies_df.write_excel(file_path)